In [1]:
from Flower import *

/home/potato/Documents/projects/master/p21cm-venv/lib/python3.11/site-packages/py21cmfast/_cfg.py:58: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn(
/home/potato/Documents/projects/master/p21cm-venv/lib/python3.11/site-packages/py21cmfast/_cfg.py:42: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn("Your configuration file is out of date. Updating...")


In [2]:
# initialize probability class

prior_ranges = {
    "global_params": {
        "M_WDM": [0.3,10.0]
    },
    "cosmo_params": {
        "OMm": [0.2,0.4]
    },
    "astro_params": {
        "L_X": [38, 42],
        "NU_X_THRESH": [100, 1500],
        "ION_Tvir_MIN": [4, 5.3],
        "HII_EFF_FACTOR": [10, 250]
    }
}

fiducial_parameter = {
    "user_params": {
        "HII_DIM": 40,
        "BOX_LEN": 160,
        "N_THREADS": 6
    },
    "flag_options": {
        "USE_TS_FLUCT": True,
        "INHOMO_RECO": True
    },
    "astro_params": {
        "INHOMO_RECO": True,
        "L_X": 40,
        "NU_X_THRESH": 500,
        "ION_Tvir_MIN": 5,
        "HII_EFF_FACTOR": 30
    },
    "cosmo_params":
        {"OMm": 0.30},
    "global_params":
        {"M_WDM": 2},
    "make_statistics": False,
}

probability = Probability(prior_ranges=prior_ranges, z_chunks=15, bins=25, debug=True, 
                          fmodel_path="./emcee_data/fiducial_ps.npy")

emcee = Flower(Probability=probability, redshift=5, data_path="./emcee_data/", noise_type=None,
                       regenerate_fiducial=False, fid_params=fiducial_parameter, debug=True)


emcee.run_emcee(threads=1)


Set initial parameters...
initialize Simulation class...
Search for existing fiducial lightcone...
Load ./emcee_data/fiducial_cone.h5 from disk...
Existing lightcone successfully loaded.
Search for existing summary statistics...
Existing summary statistics successfully loaded.
PS is  [[[2.73302457e+000 5.03920029e+000 4.20671002e+000 3.22910909e+000
   3.01673698e+000 2.61673050e+000 2.32226945e+000 2.09515135e+000
   1.98563454e+000 1.93900656e+000 1.97185790e+000 1.99076050e+000
   2.05800155e+000 2.29366312e+000 2.50131683e+000 2.66773598e+000
   2.83495225e+000 3.17497958e+000 3.62504541e+000 4.03555630e+000
   4.24713686e+000 5.00799134e+000 5.49429734e+000 7.00395124e+000
   3.62535245e+000]
  [4.21758351e-002 8.66466674e-002 1.38730260e-001 1.92808586e-001
   2.46066427e-001 3.00356959e-001 3.55571372e-001 4.09560684e-001
   4.63586660e-001 5.17701044e-001 5.72018990e-001 6.26709149e-001
   6.80738691e-001 7.35006430e-001 7.89150018e-001 8.43093414e-001
   8.97427735e-001 9.5168

/home/potato/Documents/projects/master/p21cm-venv/lib/python3.11/site-packages/py21cmfast/_cfg.py:58: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn(
/home/potato/Documents/projects/master/p21cm-venv/lib/python3.11/site-packages/py21cmfast/_cfg.py:42: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn("Your configuration file is out of date. Updating...")


Current parameters are:{'global_params': {'M_WDM': 4.114996674449708}, 'cosmo_params': {'OMm': 0.26165231188064064}, 'astro_params': {'L_X': 39.170485328261066, 'NU_X_THRESH': 1096.83810217511, 'ION_Tvir_MIN': 4.172364402169507, 'HII_EFF_FACTOR': 94.17208534628573}}
Begin lightcone simulation ...
Parameter successfully refreshed.
Current parameters are:
astro_params: AstroParams:
    ALPHA_ESC       : -0.5
    ALPHA_STAR      : 0.5
    ALPHA_STAR_MINI : 0.5
    A_LW            : 2.0
    A_VCB           : 1.0
    BETA_LW         : 0.6
    BETA_VCB        : 1.8
    F_ESC10         : 0.1
    F_ESC7_MINI     : 0.01
    F_H2_SHIELD     : 0.0
    F_STAR10        : 0.05011872336272722
    F_STAR7_MINI    : 0.01
    HII_EFF_FACTOR  : 94.17208534628573
    ION_Tvir_MIN    : 14871.82965074087
    L_X             : 1.4807622299396643e+39
    L_X_MINI        : 1e+40
    M_TURN          : 501187233.6272715
    NU_X_THRESH     : 1096.83810217511
    N_RSD_STEPS     : 20
    R_BUBBLE_MAX    : 50.0
  

/home/potato/Documents/projects/master/p21cm-venv/lib/python3.11/site-packages/py21cmfast/inputs.py:515: UserWarning: The USE_INTERPOLATION_TABLES setting has changed in v3.1.2 to be default True. You can likely ignore this warning, but if you relied onhaving USE_INTERPOLATION_TABLES=False by *default*, please set it explicitly. To silence this warning, set it explicitly to True. Thiswarning will be removed in v4.
  warnings.warn(


Done simulating. Do sanity check...
Sanity check passed. Write statistics...
Statistics written. Do filtering...
Filtering passed. Save or return file now.
Compute 1D PS using int chunks
PS is [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.] for bin 0
for ks [0.04105011 0.08576779 0.13847067 0.19263562 0.24651282 0.30033859
 0.35410529 0.40718555 0.46116041 0.51556668 0.56958613 0.62359855
 0.6774705  0.73153017 0.78565596 0.83925894 0.89288309 0.94691118
 1.000412   1.05387267 1.10728275 1.16166795 1.21531997 1.26683305
 1.31598274]
PS is [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.] for bin 1
for ks [0.04105011 0.08576779 0.13847067 0.19263562 0.24651282 0.30033859
 0.35410529 0.40718555 0.46116041 0.51556668 0.56958613 0.62359855
 0.6774705  0.73153017 0.78565596 0.83925894 0.89288309 0.94691118
 1.000412   1.05387267 1.10728275 1.16166795 1.21531997 1.26683305
 1.31598274]
PS is [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

/home/potato/Documents/projects/master/21cm-wrapper/Flower.py:74: RuntimeWarning: overflow encountered in multiply
  divide = np.divide(np.multiply(diff, diff), np.abs(fiducial_lc)+1)
/home/potato/Documents/projects/master/p21cm-venv/lib/python3.11/site-packages/py21cmfast/_cfg.py:58: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn(
/home/potato/Documents/projects/master/p21cm-venv/lib/python3.11/site-packages/py21cmfast/_cfg.py:42: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn("Your configuration file is out of date. Updating...")


Current parameters are:{'global_params': {'M_WDM': 9.937516605759651}, 'cosmo_params': {'OMm': 0.3351523507752781}, 'astro_params': {'L_X': 41.04619445676955, 'NU_X_THRESH': 1409.9417855582421, 'ION_Tvir_MIN': 4.0184599529451095, 'HII_EFF_FACTOR': 191.1715794751016}}
Begin lightcone simulation ...
Parameter successfully refreshed.
Current parameters are:
astro_params: AstroParams:
    ALPHA_ESC       : -0.5
    ALPHA_STAR      : 0.5
    ALPHA_STAR_MINI : 0.5
    A_LW            : 2.0
    A_VCB           : 1.0
    BETA_LW         : 0.6
    BETA_VCB        : 1.8
    F_ESC10         : 0.1
    F_ESC7_MINI     : 0.01
    F_H2_SHIELD     : 0.0
    F_STAR10        : 0.05011872336272722
    F_STAR7_MINI    : 0.01
    HII_EFF_FACTOR  : 191.1715794751016
    ION_Tvir_MIN    : 10434.219125301535
    L_X             : 1.1122296202436658e+41
    L_X_MINI        : 1e+40
    M_TURN          : 501187233.6272715
    NU_X_THRESH     : 1409.9417855582421
    N_RSD_STEPS     : 20
    R_BUBBLE_MAX    : 50.

/home/potato/Documents/projects/master/p21cm-venv/lib/python3.11/site-packages/py21cmfast/inputs.py:515: UserWarning: The USE_INTERPOLATION_TABLES setting has changed in v3.1.2 to be default True. You can likely ignore this warning, but if you relied onhaving USE_INTERPOLATION_TABLES=False by *default*, please set it explicitly. To silence this warning, set it explicitly to True. Thiswarning will be removed in v4.
  warnings.warn(


In [2]:
test = np.array([[2.00090386e+00, 4.20478513e+00, 3.39876988e+00, 2.46556551e+00,
        2.26569528e+00, 1.89322332e+00, 1.62326851e+00, 1.41823733e+00,
        1.32057172e+00, 1.27925759e+00, 1.30834774e+00, 1.32512361e+00,
        1.38501250e+00, 1.59727644e+00, 1.78692366e+00, 1.94038374e+00,
        2.09571169e+00, 2.41450171e+00, 2.84125951e+00, 3.23414409e+00,
        3.43771698e+00, 4.17443632e+00, 4.64827858e+00, 6.12888953e+00,
        2.84155219e+00],
       [4.41426346e+00, 1.52235711e+01, 1.07159473e+01, 8.41774134e+00,
        8.95201970e+00, 7.05721333e+00, 5.85292387e+00, 5.93059535e+00,
        5.41898020e+00, 5.09776388e+00, 5.11351888e+00, 5.15826223e+00,
        5.45265403e+00, 5.64687539e+00, 6.08552610e+00, 6.38329765e+00,
        6.98858140e+00, 7.73044592e+00, 8.46065384e+00, 9.66579972e+00,
        1.07733201e+01, 1.19233024e+01, 1.28458272e+01, 1.53642886e+01,
        1.81607708e+01],
       [8.18112997e+00, 1.56874548e+01, 1.57745141e+01, 1.53653584e+01,
        1.35180978e+01, 1.34480690e+01, 1.20189905e+01, 1.09335888e+01,
        1.04758563e+01, 9.75000081e+00, 9.21166517e+00, 9.66932040e+00,
        9.78645108e+00, 1.03466191e+01, 1.06370470e+01, 1.10586382e+01,
        1.20641385e+01, 1.25550303e+01, 1.44663600e+01, 1.53948108e+01,
        1.72318969e+01, 1.77760941e+01, 1.99179058e+01, 2.15782355e+01,
        2.10816176e+01],
       [1.59505277e+01, 5.40984943e+00, 1.36269339e-01, 9.74644605e-01,
        1.29061277e+00, 2.78313733e+00, 4.13446170e+00, 3.18854647e+00,
        3.88685070e+00, 3.05696431e+00, 3.10934104e+00, 2.54795741e+00,
        2.24450536e+00, 2.33806107e+00, 2.28885685e+00, 2.10923548e+00,
        1.87162424e+00, 1.29892233e+00, 1.12805847e+00, 1.01058114e+00,
        5.86626390e-01, 8.83311806e-02, 1.70528308e+00, 1.37664558e+00,
        2.23954694e-01],
       [6.12727418e+03, 1.17209196e+04, 1.53511986e+03, 1.09273637e+03,
        7.09752043e+02, 6.58096087e+02, 4.16722625e+02, 3.16837140e+02,
        3.43659776e+02, 2.75497530e+02, 3.17292908e+02, 3.42747479e+02,
        3.27238619e+02, 4.47032144e+02, 4.25061631e+02, 5.21663473e+02,
        5.94012929e+02, 7.10245422e+02, 8.41372565e+02, 1.01430959e+03,
        1.14910389e+03, 1.34133805e+03, 1.35198180e+03, 1.11531613e+03,
        3.87949502e+03],
       [3.61650520e+04, 5.08010878e+04, 5.25325548e+04, 4.49547307e+04,
        2.92177286e+04, 2.24911716e+04, 2.11663205e+04, 1.62044092e+04,
        1.57624123e+04, 1.51070804e+04, 1.31365668e+04, 1.36022969e+04,
        1.43021377e+04, 1.42708512e+04, 1.56313105e+04, 1.66401414e+04,
        1.95419734e+04, 2.20008649e+04, 2.58960983e+04, 2.97014129e+04,
        3.25403635e+04, 3.21446687e+04, 3.66948698e+04, 3.49308240e+04,
        3.48104747e+04],
       [5.92327047e+03, 5.36916903e+03, 1.57416261e+04, 4.03751106e+04,
        3.96642692e+04, 8.36886132e+04, 8.75631707e+04, 8.39741383e+04,
        7.63948486e+04, 8.15606021e+04, 7.92251545e+04, 6.56921969e+04,
        5.97548161e+04, 6.87525757e+04, 8.59175669e+04, 1.30557609e+05,
        1.30253744e+05, 1.38796055e+05, 1.60734694e+05, 1.75766915e+05,
        1.95834359e+05, 2.10392077e+05, 3.16205775e+05, 2.14378590e+05,
        5.26896187e+05],
       [5.34934842e+01, 3.76594722e+00, 4.56133996e+02, 3.00856665e+03,
        4.10673814e+03, 9.45850387e+03, 1.12385811e+04, 1.51866596e+04,
        1.34201851e+04, 1.57723994e+04, 1.58312515e+04, 1.48972526e+04,
        1.28031488e+04, 1.55000048e+04, 2.38342871e+04, 4.15443309e+04,
        4.59961782e+04, 5.69903419e+04, 6.54580508e+04, 8.52931553e+04,
        9.21372207e+04, 1.18272427e+05, 1.41815608e+05, 8.67644645e+04,
        1.20758218e+05],
       [2.93973142e+02, 4.72147388e+01, 3.22311752e+00, 8.91687352e+01,
        1.91160333e+02, 4.89669608e+02, 5.00074545e+02, 5.90639716e+02,
        6.32343784e+02, 7.91501106e+02, 8.41320493e+02, 8.31443805e+02,
        7.88333004e+02, 1.09162605e+03, 1.45917672e+03, 2.35274582e+03,
        2.61865278e+03, 3.08017491e+03, 3.37418011e+03, 3.89029058e+03,
        4.92676301e+03, 6.09572344e+03, 9.35057206e+03, 6.54789203e+03,
        1.45177420e+04],
       [2.17970663e+03, 1.32220524e+03, 9.17664636e+02, 4.14510023e+02,
        1.87901160e+02, 2.90618638e+02, 2.57614341e+02, 1.89523313e+02,
        1.12106381e+02, 1.74884136e+02, 7.62798286e+01, 9.93354120e+01,
        1.61133459e+02, 1.40801905e+02, 1.17493952e+02, 1.00640674e+02,
        1.24280894e+02, 1.55560832e+02, 1.95898184e+02, 2.46795845e+02,
        3.27694930e+02, 4.17115373e+02, 4.33928476e+02, 7.13972192e+02,
        7.26155078e+02],
       [1.71642031e+00, 4.62225111e+01, 4.59083841e+00, 6.96597531e+01,
        1.56280319e+01, 1.38987857e+01, 2.41514218e+01, 1.27511126e+01,
        8.26822562e+00, 4.88142368e+00, 1.35412527e+01, 5.99696515e+00,
        5.01066799e+00, 5.17753563e+00, 4.45413653e+00, 3.02995819e+00,
        2.00906048e+00, 1.97770474e+00, 1.43696765e+00, 2.30864606e+00,
        3.78284650e+00, 5.29259560e+00, 1.25388065e+01, 1.10334847e+01,
        2.03283486e+02],
       [1.00313197e+00, 5.61775248e-01, 2.12582479e-01, 8.84091752e-03,
        1.13772516e-01, 8.83771146e-02, 1.90207385e-02, 1.32475783e-01,
        1.13114904e-01, 3.11493774e-01, 8.42204422e-02, 1.00086654e-01,
        8.33123787e-02, 6.58249953e-02, 1.09921519e-01, 4.89632741e-01,
        4.44039544e-01, 3.88072305e-01, 2.17387125e-01, 1.68472583e-01,
        1.78796235e-01, 4.83749035e-02, 1.90882811e-01, 1.75764204e-01,
        2.07510932e-01],
       [1.73016009e-02, 6.35283196e-03, 1.73321834e-02, 3.69106293e-02,
        9.37880288e-02, 1.39448236e-01, 2.13138522e-01, 2.33995683e-01,
        2.73975304e-01, 3.39946600e-01, 3.50358788e-01, 3.39274598e-01,
        2.97629796e-01, 3.15479972e-01, 3.15641000e-01, 2.65821856e-01,
        1.88070920e-01, 1.79622042e-01, 1.50627163e-01, 1.45243128e-01,
        9.35412982e-02, 5.72168040e-02, 1.91314359e-02, 3.72579351e-03,
        2.13772154e-01],
       [1.16810866e-03, 1.18291140e-04, 3.49873190e-04, 2.33140668e-04,
        1.39498955e-03, 6.58061962e-06, 6.62312573e-05, 7.57823467e-06,
        3.08702256e-04, 1.56331888e-08, 8.85579195e-05, 1.78522628e-05,
        3.86612312e-05, 4.81184786e-05, 4.51555117e-05, 2.10617566e-05,
        2.00330199e-05, 2.49146382e-05, 2.83996451e-05, 3.70683257e-05,
        4.17348847e-05, 4.73131072e-05, 4.87271906e-05, 6.81502357e-05,
        6.82637622e-05],
       [4.88195968e-03, 8.72756061e-04, 6.16803949e-03, 3.03995082e-02,
        5.98863427e-03, 1.87814779e-03, 1.23299644e-02, 1.11191513e-03,
        3.30616376e-04, 6.98996414e-03, 1.78627300e-05, 4.11776317e-04,
        1.66527723e-03, 5.26139809e-04, 1.91839175e-04, 2.29328383e-06,
        4.88470810e-07, 1.36215337e-06, 2.41691481e-06, 2.63962736e-07,
        7.27951720e-07, 4.22581394e-07, 2.94958891e-09, 1.49631944e-07,
        6.91476098e-07],
       [3.41355094e+02, 3.42897116e+02, 3.44447586e+02, 3.46006572e+02,
        3.47574121e+02, 3.49150285e+02, 3.50735138e+02, 3.52328719e+02,
        3.53931097e+02, 3.55542328e+02, 3.57162463e+02, 3.58791568e+02,
        3.60429695e+02, 3.62076908e+02, 3.63733264e+02, 3.65398835e+02,
        3.67073661e+02, 3.68757827e+02, 3.70451379e+02, 3.72154384e+02,
        3.73866905e+02, 3.75588995e+02, 3.77320741e+02, 3.79062187e+02,
        3.80813400e+02]])

In [6]:
np.log(np.sum(test))

15.399323637922153